#### Funções

In [18]:
import pandas as pd
from collections import Counter
import random
from utils.Gpt_generator import Gpt_generator
from utils.Json_controller import Json_controller
import time

In [19]:
def get_data(dir= 'data', split= 'train', track= 'a', language= 'ptbr'):
    
    archive = language + '.csv' if split == 'train' else language + '_' + track + '.csv'

    path = f'{dir}/{split}/track_{track}/{archive}'
    
    return pd.read_csv(path)

## Abordagem 1

Vamos contemplar, primeiramente, a geração de dados sintéticos para a track A, onde temos uma classificação binária de emoções.

Seguiremos a abordagem do artigo: https://aclanthology.org/2024.acl-long.120.pdf


Focaremos na variabilidade desses dados, e uma abordagem few shot, iremos variar na seguinte medida:


    - Emoções (faremos isso dinâmico em função do dataset (a probabilidade será proporcional a quantidade de amostras))
    - Tamanho do tweet (amostrados em função dos tamanhos reais)
    - Intensidade do sentimento (leve, moderado, forte)



Os dados serão retornados em formato de JSON

In [20]:
df = get_data()

'''Vamos estratificar o dataset, tendo somente uma coluna de emoções, isso nos ajuda, pois teremos agora também as 
combinações de emoções, posteriormente, vamos amostrar essas emoções para geração de novos dados, dando maior probabilidade a 
emoções ou combinações mais raras, atualizando isso de maneira dinâmica para atualizar a geração.'''


emotion_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
df['labels'] = df[emotion_columns].apply(
    lambda row: ' '.join([col for col, val in row.items() if val == 1]),
    axis=1
)


df['labels'] = df['labels'].apply(
    lambda x: 'Neutral' if x == '' else x
)

df = df.drop(columns=emotion_columns)

In [21]:
def categorize(length):
        if length <= 11:
            return "Pequeno"
        elif length <= 26:
            return "Médio"
        elif length >= 26:
            return "Grande"
        

'''Gera a distribuição de tamanhos de texto'''
category_counts = dict(Counter(df['text'].apply(lambda x: categorize(len(x.split())))))
'''Gera a distribuição de emoções'''
emotion_counts = dict(Counter(df['labels']))

'''Gera uma distribuição da intensidade de emoções (baseados nos dados da track b)'''
aux = get_data(track='b')
intensity_counts = {
    emotion: (aux[aux[emotion] > 0][emotion].value_counts(normalize=True) * 100).round(2).to_dict()
    for emotion in emotion_columns
}

intensity_counts = {
    emotion: {
        "baixo": values.get(1, 0), 
        "moderado": values.get(2, 0),
        "alto": values.get(3, 0)
    }
    for emotion, values in intensity_counts.items()
}

In [22]:
def amostra_categoria_e_emocao(label_dict, category_dict, intensity_counts):
    '''Amostraremos baseado no tamanhjo do texto, emoção e intensidade da emoção.
    
    Para o tamanho e emoção, temos um dicionario dinâmico, onde as probabilidades da amostragem de cada emoção mudam ao longo
    do tempo, baseado na quantidade de vezes que cada emoção foi amostrada. Para a intensidade, faremos de maneira estática, visto que
    dados menos comuns são dados extremos (emoção alta), o que é muito fácil de se identificar no caso da track A. Por isso, usaremos como
    probabilidade a presença de cada intensidade em cada emoção nos dados da track b, adicionamos essa "feature" apenas para gerar mais 
    variabilidade.
    '''
    inverse_weights_labels = {label: 1 / count for label, count in label_dict.items() if count > 0}
    total_weight_labels = sum(inverse_weights_labels.values())
    probabilities_labels = {label: weight / total_weight_labels for label, weight in inverse_weights_labels.items()}
    labels = list(probabilities_labels.keys())
    weights_labels = list(probabilities_labels.values())
    sampled_label = random.choices(labels, weights=weights_labels, k=1)[0]
    label_dict[sampled_label] += 1

    inverse_weights_categories = {category: 1 / count for category, count in category_dict.items() if count > 0}
    total_weight_categories = sum(inverse_weights_categories.values())
    probabilities_categories = {category: weight / total_weight_categories for category, weight in inverse_weights_categories.items()}
    categories = list(probabilities_categories.keys())
    weights_categories = list(probabilities_categories.values())
    sampled_category = random.choices(categories, weights=weights_categories, k=1)[0]
    category_dict[sampled_category] += 1
            
    intensity_emotion = {}
    for emotion in intensity_counts:
        if str(emotion) in sampled_label:
            probabilities = intensity_counts[str(emotion)]
            categories = list(probabilities.keys())
            weights = list(probabilities.values())
            intensity = random.choices(categories, weights=weights, k=1)[0]
            intensity_emotion[emotion] = intensity

    # Caso nenhuma intensidade seja encontrada, o dicionário ficará vazio
    if not intensity_emotion:
        intensity_emotion = {"Nenhuma": "intensidade encontrada"}

    return intensity_emotion, sampled_category

### Geração do prompt

In [23]:
prompt = """
Você é um assistente criativo especializado em criar textos sintéticos que simulam postagens informais, mas realistas,
para redes sociais. Esses textos devem refletir emoções genuínas e naturais, sem exageros ou estereótipos. 

Para criar esses textos:
- Use gírias, abreviações e expressões informais comuns no ambiente online.
- Os textos podem conter xingamentos ou linguagem inapropriada.
- Não use emojis ou hashtags, a menos que sejam essenciais para a naturalidade do texto.
- Insira as emoções indicadas com as intensidades especificadas de maneira criativa, mas coerente com o texto. Não é necessário usar as mesmas palavras das emoções, mas elas devem ser perceptíveis no contexto.
- Atenda ao tamanho do texto solicitado.

Exemplos:
Os exemplos abaixo mostram o estilo esperado. Leia com atenção para entender como transmitir as emoções de forma autêntica:

{exemplos}

Sua Tarefa:
- Gerar um texto em português seguindo o estilo dos exemplos fornecidos.
- Atender às seguintes características:
  - Emoções PARA GERAR (IMPORTANTE): {emoções}
  - Tamanho do texto: {tamanho}
- Certifique-se de que as emoções aparecem no texto nas intensidades indicadas, mas sem tornar isso explícito ou forçado.

Regras Importantes:
1. Não copie os exemplos diretamente. Use sua criatividade para criar algo original.
2. O texto deve parecer uma postagem real, como se fosse escrito por uma pessoa comum.
3. Evite clichês ou exageros que possam tornar o texto artificial.
4. Foque na linguagem e tom apropriados para um post informal, mas autêntico.

Gere o texto solicitado abaixo:
"""


def sample_data_for_emotions(df, emotion_intensity):
    required_emotions = set(emotion_intensity.keys())
    def contains_all_emotions(label):
        label_emotions = set(label.split())
        return required_emotions.issubset(label_emotions)
    filtered_df = df[df['labels'].apply(contains_all_emotions)]

    return filtered_df


def few_shot_examples(df, emotion_intensity, n=8):
    filtered_df = sample_data_for_emotions(df, emotion_intensity)

    if filtered_df.empty:
        return "Nenhum exemplo encontrado com as emoções especificadas."

    examples = filtered_df.sample(min(n-3, len(filtered_df)), random_state=42)
    random_examples = df.sample(3, random_state=42)
    combined_examples = pd.concat([examples, random_examples]).sample(frac=1, random_state=42)

    formatted_examples = [
        f"- Emoções: {row['labels']}\n- Texto: {row['text']}"
        for _, row in combined_examples.iterrows()
    ]

    return '\n\n'.join(formatted_examples)


def get_prompt(intensity_emotion : dict, lenght : str, df : pd.DataFrame):
    #print(intensity_emotion)
    examples = few_shot_examples(df, intensity_emotion)
    emocoes = ', '.join([f"{emocao} com intensidade {intensidade}" for emocao, intensidade in intensity_emotion.items()])
    return prompt.format(exemplos=examples, tamanho=lenght, emoções=emocoes)
    

### Geração dos dados

In [24]:
from utils.Gpt_generator import Gpt_generator
llm = Gpt_generator()

In [25]:
prompt = get_prompt({'Disgust': 'baixo'}, 'Pequeno', df)

print(llm.invoke(prompt))

Sério que esqueceram um chiclete grudado na minha carteira da facul? Pô, que falta de noção.


In [26]:
print(prompt)


Você é um assistente criativo especializado em criar textos sintéticos que simulam postagens informais, mas realistas,
para redes sociais. Esses textos devem refletir emoções genuínas e naturais, sem exageros ou estereótipos. 

Para criar esses textos:
- Use gírias, abreviações e expressões informais comuns no ambiente online.
- Os textos podem conter xingamentos ou linguagem inapropriada.
- Não use emojis ou hashtags, a menos que sejam essenciais para a naturalidade do texto.
- Insira as emoções indicadas com as intensidades especificadas de maneira criativa, mas coerente com o texto. Não é necessário usar as mesmas palavras das emoções, mas elas devem ser perceptíveis no contexto.
- Atenda ao tamanho do texto solicitado.

Exemplos:
Os exemplos abaixo mostram o estilo esperado. Leia com atenção para entender como transmitir as emoções de forma autêntica:

- Emoções: Disgust
- Texto: nao acredito que meu pao mofou de novo que nojo

- Emoções: Anger Disgust
- Texto: Fortalece a esquerd

#### GERANDO E SALVANDO

In [28]:
json_file = 'dados_sinteticos.json'
json_controller = Json_controller(json_file)
data = json_controller.load_data()

In [29]:
n_examples = 10000

In [31]:
i = len(data)
for n in range(n_examples):    
    intensity_emotion, sampled_category = amostra_categoria_e_emocao(emotion_counts, category_counts, intensity_counts) 
    
    print(f"Generating example {n+1}/{n_examples}   -  EMOTION: {intensity_emotion}    -   CATEGORY: {sampled_category}")

    
    prompt = get_prompt(intensity_emotion, sampled_category, df)
    
    text = llm.invoke(prompt)
    
    example_id = f'synthetic_{n}'
    
    emotions = { "Anger": 0, "Disgust": 0, "Fear": 0, "Joy": 0, "Sadness": 0, "Surprise": 0 }
    
    for emotion, intensity in intensity_emotion.items():
        if intensity in ["baixo", "moderado", "alto"]:
            emotions[emotion] = 1
        
        
    entry = {
        "id": example_id,
        "text": text,
        **emotions
    }
    
    data[str(n)] = entry
    json_controller.save_data(data)
    
    time.sleep(4)

Generating example 1/10000   -  EMOTION: {'Disgust': 'baixo', 'Surprise': 'baixo'}    -   CATEGORY: Grande
Generating example 2/10000   -  EMOTION: {'Disgust': 'baixo', 'Sadness': 'baixo', 'Surprise': 'baixo'}    -   CATEGORY: Médio
Generating example 3/10000   -  EMOTION: {'Joy': 'moderado', 'Sadness': 'baixo', 'Surprise': 'moderado'}    -   CATEGORY: Grande
Generating example 4/10000   -  EMOTION: {'Anger': 'baixo', 'Disgust': 'baixo', 'Surprise': 'baixo'}    -   CATEGORY: Médio
Generating example 5/10000   -  EMOTION: {'Disgust': 'baixo'}    -   CATEGORY: Médio
Generating example 6/10000   -  EMOTION: {'Anger': 'baixo', 'Disgust': 'baixo', 'Fear': 'alto', 'Sadness': 'moderado'}    -   CATEGORY: Grande
Generating example 7/10000   -  EMOTION: {'Disgust': 'baixo', 'Sadness': 'baixo', 'Surprise': 'baixo'}    -   CATEGORY: Grande
Generating example 8/10000   -  EMOTION: {'Disgust': 'baixo', 'Fear': 'baixo'}    -   CATEGORY: Pequeno
Generating example 9/10000   -  EMOTION: {'Disgust': 'b

Exception: Erro ao invocar o modelo: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [ ]:
#todo: criar loop para geração dos dados
#todo estruturar prompt
#todo lidar com json de saída
#todo salvar dados e subir no hf

In [26]:
intensity_emotion

{'Disgust': 'baixo', 'Fear': 'baixo'}